
# ADIOS2-Python-High-Level-nompi: Write Read Cooling Rod 1D



## Writer: Generate Data
    
    We use a simple solution to the heat transfer equation in a (very-fast) 1D cooling rod.
    

In [ ]:
import math
import numpy as np
import adios2


def calcUxt(Uo, x, t):
    return 4 * Uo / math.pi * math.sin(math.pi * x) * math.exp(- math.pi * math.pi * t)

# Initialize local variable dimensions
nx = 10

shape = [nx]
start = [0]
count = [nx]

# Initialize problem x cell coord, Uo, Uxt
steps = 5
Lx = 1. 
Dx = 1./nx

x = np.full( (nx), 0. )
for i in range(0,nx):
    x[i] = (i+0.5) * Dx

# Make sure you use floats
Uo = 1. 
Uxt = np.full( (nx), Uo)

# Writer save U solution as function of location (x) and time (t)
with adios2.open("Uxt.bp", "w") as fw:

    for t in range(0, steps):
        # We save "x" only once at step "0"
        if(t == 0):
            fw.write("x", x, shape, start, count)
        else:
            for i in range(0,nx):
                Uxt[i] = calcUxt(Uo, x[i], t)
                
        fw.write("Uxt", Uxt, shape, start, count)
        fw.end_step()

The data is saved in a bp file and subsfile directory structure. To access the data, the user doesn't need to manipulate these directly, but rather using the **bpls** command line utility for quick inspection. For example:

In [ ]:
!bpls Uxt.bp -lavd -n 10

## Reader: Accessing and Understanding the Data

In [ ]:
import math
import numpy as np
import adios2
import matplotlib.pyplot as plt


Uxt_plots = plt.subplots()

with adios2.open("Uxt.bp", "r") as fr:

    for fr_step in fr:

        step = fr_step.current_step()
        if(step == 0):
            x = fr_step.read("x")
            
        Uxt = fr_step.read("Uxt")
        
        plt.semilogy(x, Uxt, label = "Ux for time= " + str(step))
        
legend = plt.legend(loc='best', shadow=True, fontsize='x-large')